<h1> Cloud ML Engineを利用して機械学習をスケールアップする </h1>

このnotebookでは、これまで作成してきたタクシーの乗車料金を予測するTensorFlowのモデルをCloud ML Engineで利用できるようにパッケージ化する方法を学びます。<br>
まずは、小さなデータセットから始めていきましょう。

このコースの後半では、より効果的な機械学習モデルの作成方法について確認していきます。

<h2> プロジェクトとバケットを環境変数として入力する</h2>

注意事項:
<ol>
<li> Project IDは使用するプロジェクトを表す*ユニーク*な文字列です（プロジェクト名とは異なります）。GCPコンソールのホーム画面から確認することができます。 </li>
<li> クラウド上での機械学習のトレーニングでは、モデルファイルを保存したり復元したりというプロセスを含んでいます。まだバケットを持っていない場合、GCPコンソールからバケットを一つ作成してください。Project IDを先頭につけることでユニークなバケット名をつけることが一般的です。またコスト上の理由から、単一のRegionでバケットを作成するのがよいでしょう。</li>
</ol>
自分のProject IDとバケット名に<b>以下のセルの変数を変更してください。</b>


In [ ]:
import os
PROJECT = 'cloud-training-demos' # ご自分のProject IDに変更してください。
REGION = 'us-central1' # AI Platformを利用できるリージョンを選択してください https://cloud.google.com/ml-engine/docs/regions
BUCKET = 'cloud-training-demos-ml' # ご自分のバケット名に変更してください。選択したリージョンのリージョナルなバケットを使用してください。

In [ ]:
# for bash
os.environ['PROJECT'] = PROJECT
os.environ['BUCKET'] = BUCKET
os.environ['REGION'] = REGION
os.environ['TFVERSION'] = '1.4'  # Tensorflow version

In [ ]:
%%bash
gcloud config set project $PROJECT
gcloud config set compute/region $REGION

トレーニングデータを含むバケットの読み書きができるように、Cloud ML Engineサービスアカウントを認証します。

In [ ]:
%%bash
PROJECT_ID=$PROJECT
AUTH_TOKEN=$(gcloud auth print-access-token)
SVC_ACCOUNT=$(curl -X GET -H "Content-Type: application/json" \
    -H "Authorization: Bearer $AUTH_TOKEN" \
    https://ml.googleapis.com/v1/projects/${PROJECT_ID}:getConfig \
    | python -c "import json; import sys; response = json.load(sys.stdin); \
    print(response['serviceAccount'])")

echo "Authorizing the Cloud ML Service account $SVC_ACCOUNT to access files in $BUCKET"
gsutil -m defacl ch -u $SVC_ACCOUNT:R gs://$BUCKET
gsutil -m acl ch -u $SVC_ACCOUNT:R -r gs://$BUCKET  # error message (if bucket is empty) can be ignored
gsutil -m acl ch -u $SVC_ACCOUNT:W gs://$BUCKET

<h2> コードをパッケージ化する</h2>

コードをPythonの標準的なパッケージ構造にしましょう。<a href="taxifare/trainer/model.py">model.py</a>と<a href="taxifare/trainer/task.py">task.py</a>にはこれまでに扱ったTensorFlowコードが格納されています。taxifare/trainerディレクトリの中の構造を見てみましょう。

In [ ]:
!find taxifare

In [ ]:
!cat taxifare/trainer/model.py

<h2> データへの絶対パスを確認する</h2>

以下の絶対パスに注目してください。

In [ ]:
%%bash
echo $PWD
rm -rf $PWD/taxi_trained
cp $PWD/../tensorflow/taxi-train.csv .
cp $PWD/../tensorflow/taxi-valid.csv .
head -1 $PWD/taxi-train.csv
head -1 $PWD/taxi-valid.csv

<h2> Pythonモジュールをコマンドラインから実行する</h2>

In [ ]:
%%bash
rm -rf taxifare.tar.gz taxi_trained
export PYTHONPATH=${PYTHONPATH}:${PWD}/taxifare
python -m trainer.task \
   --train_data_paths="${PWD}/taxi-train*" \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --output_dir=${PWD}/taxi_trained \
   --train_steps=1000 --job-dir=./tmp

In [ ]:
%%bash
ls $PWD/taxi_trained/export/exporter/

In [ ]:
%%writefile ./test.json
{"pickuplon": -73.885262,"pickuplat": 40.773008,"dropofflon": -73.987232,"dropofflat": 40.732403,"passengers": 2}

In [ ]:
## local predict doesn't work with Python 3 yet
#%bash
#model_dir=$(ls ${PWD}/taxi_trained/export/exporter)
#gcloud ai-platform local predict \
#    --model-dir=${PWD}/taxi_trained/export/exporter/${model_dir} \
#    --json-instances=./test.json

<h2> gcloudコマンドを利用してローカルで実行する</h2>

In [ ]:
%%bash
rm -rf taxifare.tar.gz taxi_trained
gcloud ai-platform local train \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   -- \
   --train_data_paths=${PWD}/taxi-train.csv \
   --eval_data_paths=${PWD}/taxi-valid.csv  \
   --train_steps=1000 \
   --output_dir=${PWD}/taxi_trained 

ランダムシードの影響で結果は異なりますが、私が実行した際には```average_loss```は187程度になりました。つまり、RMSEは13前後となります。

以下のステップでTensorboardで学習の過程を確認してみましょう。
<ol>
<li>左上のファイルアイコン -> "+" をクリックする
<li>Tensorboardをクリックする
</ol>

In [ ]:
!ls $PWD/taxi_trained

<h2> gcloudコマンドで学習ジョブを送信する</h2>

はじめに、学習データをクラウドにコピーしましょう。コピーができたら学習ジョブを実行します。

ジョブを送信できたら、コンソールから <b>AI Platform -> Jobs</b>に移動し、ジョブの進捗をモニタリングしましょう

<b>注意：</b>notebookのセルが実行中（青のプログレスバーが表示される）で止まったり、認証トークンのリフレッシュができないなどのエラーが返ってきても気にしないでください。<br>
ジョブ自体はクラウド上で実行されているので、コンソールからジョブのモニタリングを行ってください


In [ ]:
%%bash
echo $BUCKET
gsutil -m rm -rf gs://${BUCKET}/taxifare/smallinput/
gsutil -m cp ${PWD}/*.csv gs://${BUCKET}/taxifare/smallinput/

In [ ]:
%%bash
OUTDIR=gs://${BUCKET}/taxifare/smallinput/taxi_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=BASIC \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${BUCKET}/taxifare/smallinput/taxi-train*" \
   --eval_data_paths="gs://${BUCKET}/taxifare/smallinput/taxi-valid*"  \
   --output_dir=$OUTDIR \
   --train_steps=10000

<h2> モデルをデプロイする </h2>

モデルが保存されいるディレクトリを指定して、モデルのデプロイに使用しましょう。モデルのデプロイには<b>5分</b>ほどかかる場合があります。


In [ ]:
%%bash
gsutil ls gs://${BUCKET}/taxifare/smallinput/taxi_trained/export/exporter

In [ ]:
%%bash
MODEL_NAME="taxifare"
MODEL_VERSION="v1"
MODEL_LOCATION=$(gsutil ls gs://${BUCKET}/taxifare/smallinput/taxi_trained/export/exporter | tail -1)
echo "Run these commands one-by-one (the very first time, you'll create a model and then create a version)"
#gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
#gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions $REGION
gcloud ai-platform versions create ${MODEL_VERSION} --model ${MODEL_NAME} --origin ${MODEL_LOCATION} --runtime-version $TFVERSION

### Tensorboardを開く

<ol>
<li>以下を実行し、出力をCloud Shellで実行する
<li>Cloud Shell上のWeb Previewをクリックする
<li>Preview on port 8080をクリックする
</ol>


In [ ]:
OUTDIR='gs://{0}/taxifare/smallinput/taxi_trained'.format(BUCKET)
print('tensorboard --port=8080 --logdir={0}'.format(OUTDIR))

<h2> 予測を行う</h2>

In [ ]:
%%bash
gcloud ai-platform predict --model=taxifare --version=v1 --json-instances=./test.json

In [ ]:
from googleapiclient import discovery
from oauth2client.client import GoogleCredentials
import json

credentials = GoogleCredentials.get_application_default()
api = discovery.build('ml', 'v1', credentials=credentials,
            discoveryServiceUrl='https://storage.googleapis.com/cloud-ml/discovery/ml_v1_discovery.json')

request_data = {'instances':
  [
      {
        'pickuplon': -73.885262,
        'pickuplat': 40.773008,
        'dropofflon': -73.987232,
        'dropofflat': 40.732403,
        'passengers': 2,
      }
  ]
}

parent = 'projects/%s/models/%s/versions/%s' % (PROJECT, 'taxifare', 'v1')
response = api.projects().predict(body=request_data, name=parent).execute()
print("response={0}".format(response))

<h2> 大規模なデータセットで学習を行う</h2>

以下のステップはすでに実行済みでデータセットのファイルを利用することができますの。そのため<b>以下のコメントに記載されているステップを実行する必要はありません。</b><br>
次の章（特徴量エンジニアリング）では、同様の前処理をCloud Dataflowを用いて行います。

http://bigquery.cloud.google.com/ に行って以下のクエリを入力してください

<pre>
SELECT
  (tolls_amount + fare_amount) AS fare_amount,
  pickup_longitude AS pickuplon,
  pickup_latitude AS pickuplat,
  dropoff_longitude AS dropofflon,
  dropoff_latitude AS dropofflat,
  passenger_count*1.0 AS passengers,
  'nokeyindata' AS key
FROM
  [nyc-tlc:yellow.trips]
WHERE
  trip_distance > 0
  AND fare_amount >= 2.5
  AND pickup_longitude > -78
  AND pickup_longitude < -70
  AND dropoff_longitude > -78
  AND dropoff_longitude < -70
  AND pickup_latitude > 37
  AND pickup_latitude < 45
  AND dropoff_latitude > 37
  AND dropoff_latitude < 45
  AND passenger_count > 0
  AND ABS(HASH(pickup_datetime)) % 1000 == 1
</pre>

100万行のデータを取得することに注意してください。これを以下のステップでCSVファイルとしてエクスポートします。<br>
（<b>このステップはすでに実行済みで、結果はGCSでパブリックにアクセスできるようになっています。</b>そのため実行する必要はありません）

<ol>
<li> "Save As Table"ボタンをクリックし、データセット名とテーブル名を指定します。
<li> BigQueryコンソール上で、左側のメニューから新しく追加されたテーブルを見つけ、クリックします。
<li> "Export Table"をクリックします
<li> バケット名を指定し、train.csvとファイル名を指定します。 (例: gs://cloud-training-demos-ml/taxifare/ch3/train.csv). ジョブが完了するのを待ちます。（左側のメニューの"Job History"を見てみましょう。）
<li> 上のクエリから、最後の"== 1"を"== 2"に変更し、Cloud Storageにvalid.csvというファイル名でエクスポートしましょう (e.g.  gs://cloud-training-demos-ml/taxifare/ch3/valid.csv)
<li> 2つのファイルをダウンロードし、ヘッダー行を削除して再アップロードします。
</ol>

<p/>
<p/>

<h2> 100万行のデータセットを使用してクラウド上での学習を行う</h2>

100万行のデータの学習には60分ほど時間がかかります。モデルは先程のものと全く同じものです。唯一の違いは入力（大規模なデータセットを利用）と、Cloud ML Engineのtier（BASICではなくSTANDARD_1を利用します。STANDARD_1はBASICのおよそ10倍の性能があります）です。<br>
学習の最後のlossは32ですが、RMSE（検証データセットに対して算出）は変わらず9.03です。つまり、単にデータを足すだけではモデルの精度は上がらないようです。

In [ ]:
%%bash

## this takes 60 minutes. 

OUTDIR=gs://${BUCKET}/taxifare/ch3/taxi_trained
JOBNAME=lab3a_$(date -u +%y%m%d_%H%M%S)
CRS_BUCKET=cloud-training-demos # use the already exported data
echo $OUTDIR $REGION $JOBNAME
gsutil -m rm -rf $OUTDIR
gcloud ai-platform jobs submit training $JOBNAME \
   --region=$REGION \
   --module-name=trainer.task \
   --package-path=${PWD}/taxifare/trainer \
   --job-dir=$OUTDIR \
   --staging-bucket=gs://$BUCKET \
   --scale-tier=STANDARD_1 \
   --runtime-version=$TFVERSION \
   -- \
   --train_data_paths="gs://${CRS_BUCKET}/taxifare/ch3/train.csv" \
   --eval_data_paths="gs://${CRS_BUCKET}/taxifare/ch3/valid.csv"  \
   --output_dir=$OUTDIR \
   --train_steps=100000

Copyright 2016 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License